In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

- 상병기호 : ['E10', 'I10', 'J00', 'J20', 'M50']

In [2]:
df = pd.read_csv('./data/hackathon.csv')
df = df.iloc[:, 1:]
df.sample(15)

,시도코드,성별코드,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,ast_alt2,감마 지티피,상병기호
7233,28,2,22.959184,103.666667,95.0,212.0,61.0,13.2,1.0,0.7,0,14.0,4
62550,11,2,21.484375,90.000000,97.0,166.0,68.0,14.6,1.0,0.9,0,16.0,3
157066,11,1,25.390625,106.000000,91.0,215.0,92.0,15.4,1.0,1.0,0,49.0,3
328074,11,1,29.296875,111.666667,102.0,291.0,219.0,17.0,1.0,1.0,0,111.0,3
276703,11,2,26.159334,85.333333,84.0,166.0,71.0,11.7,1.0,0.7,0,15.0,3
226493,30,1,27.343750,99.666667,114.0,216.0,111.0,16.3,1.0,1.0,0,20.0,3
92057,30,1,27.777778,104.000000,99.0,200.0,313.0,14.9,1.0,0.9,0,48.0,2
16050,41,1,24.221453,85.666667,90.0,300.0,134.0,14.8,1.0,0.8,0,30.0,1
144475,44,2,26.666667,98.000000,103.0,247.0,157.0,14.7,1.0,0.8,0,17.0,0
93292,41,1,27.548209,106.666667,88.0,149.0,98.0,13.9,1.0,1.0,0,31.0,3


In [3]:
# 발병 : 1 / 발병X : 0
df2 = pd.get_dummies(df['상병기호'])
df2.columns = ['E10', 'I10', 'J00', 'J20', 'M50']
df2

,E10,I10,J00,J20,M50
0,0,1,0,0,0
1,0,0,1,0,0
2,0,0,0,1,0
3,0,0,1,0,0
4,1,0,0,0,0
...,...,...,...,...,...
330953,1,0,0,0,0
330954,0,0,1,0,0
330955,0,0,0,0,1
330956,0,0,0,1,0


In [4]:
df2 = pd.concat([df.iloc[:, :-1], df2], axis=1)
df2

,시도코드,성별코드,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,ast_alt2,감마 지티피,E10,I10,J00,J20,M50
0,27,1,24.835646,86.000000,86.0,234.0,183.0,16.4,1.0,1.1,0,26.0,0,1,0,0,0
1,44,1,29.384757,86.000000,250.0,119.0,265.0,15.7,1.0,0.7,0,35.0,0,0,1,0,0
2,44,1,29.384757,86.000000,250.0,119.0,265.0,15.7,1.0,0.7,0,35.0,0,0,0,1,0
3,41,2,22.222222,88.666667,104.0,177.0,61.0,13.3,1.0,0.7,0,39.0,0,0,1,0,0
4,41,2,22.892820,89.333333,92.0,152.0,175.0,13.0,1.0,0.7,0,12.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330953,11,2,27.055151,102.333333,99.0,174.0,82.0,15.3,1.0,0.7,0,16.0,1,0,0,0,0
330954,30,1,29.387755,89.333333,98.0,142.0,121.0,15.1,1.0,0.8,0,36.0,0,0,1,0,0
330955,41,2,26.159334,96.000000,124.0,178.0,301.0,13.6,1.0,0.6,0,18.0,0,0,0,0,1
330956,26,2,21.484375,76.000000,98.0,288.0,93.0,14.0,1.0,0.8,0,19.0,0,0,0,1,0


# 전처리

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   시도코드        330958 non-null  int64  
 1   성별코드        330958 non-null  int64  
 2   bmi         330958 non-null  float64
 3   map         330958 non-null  float64
 4   식전혈당(공복혈당)  330958 non-null  float64
 5   총 콜레스테롤     330958 non-null  float64
 6   트리글리세라이드    330958 non-null  float64
 7   혈색소         330958 non-null  float64
 8   요단백         330958 non-null  float64
 9   혈청크레아티닌     330958 non-null  float64
 10  ast_alt2    330958 non-null  int64  
 11  감마 지티피      330958 non-null  float64
 12  E10         330958 non-null  uint8  
 13  I10         330958 non-null  uint8  
 14  J00         330958 non-null  uint8  
 15  J20         330958 non-null  uint8  
 16  M50         330958 non-null  uint8  
dtypes: float64(9), int64(3), uint8(5)
memory usage: 31.9 MB


In [6]:
temp = df2.select_dtypes(include=['int64', 'uint8']).astype('category')

temp2 = df2.select_dtypes(include=['float64']).astype('float32')

scaler = StandardScaler()
temp2 = pd.DataFrame(scaler.fit_transform(temp2), columns=temp2.columns) #np -> pd

df3 = pd.concat([temp2, temp], axis=1)
df3.info()
display(df3[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330958 entries, 0 to 330957
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   bmi         330958 non-null  float32 
 1   map         330958 non-null  float32 
 2   식전혈당(공복혈당)  330958 non-null  float32 
 3   총 콜레스테롤     330958 non-null  float32 
 4   트리글리세라이드    330958 non-null  float32 
 5   혈색소         330958 non-null  float32 
 6   요단백         330958 non-null  float32 
 7   혈청크레아티닌     330958 non-null  float32 
 8   감마 지티피      330958 non-null  float32 
 9   시도코드        330958 non-null  category
 10  성별코드        330958 non-null  category
 11  ast_alt2    330958 non-null  category
 12  E10         330958 non-null  category
 13  I10         330958 non-null  category
 14  J00         330958 non-null  category
 15  J20         330958 non-null  category
 16  M50         330958 non-null  category
dtypes: category(8), float32(9)
memory usage: 13.9 MB


,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,감마 지티피,시도코드,성별코드,ast_alt2,E10,I10,J00,J20,M50
0,0.182520,-0.621876,-0.717914,0.831830,0.499025,1.426068,-0.253558,0.764726,-0.219147,27,1,0,0,1,0,0,0
1,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,44,1,0,0,0,1,0,0
2,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,44,1,0,0,0,0,1,0
3,-0.549760,-0.373717,-0.003174,-0.503164,-0.735804,-0.559361,-0.253558,-0.443400,0.033405,41,2,0,0,0,1,0,0
4,-0.361858,-0.311677,-0.479667,-1.088688,0.418053,-0.751499,-0.253558,-0.443400,-0.491125,41,2,0,1,0,0,0,0
5,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,30,1,0,0,0,0,1,0
6,0.338024,1.115238,-0.400252,0.574200,1.966650,1.554161,-0.253558,0.462694,-0.258001,30,1,0,0,0,1,0,0
7,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,44,2,0,0,0,0,1,0
8,1.219763,2.014815,0.115949,4.087343,1.086075,0.081100,-0.253558,-0.443400,0.013978,44,2,0,0,0,1,0,0
9,-0.944999,-0.559836,-0.400252,1.300249,-0.492887,-1.520052,-0.253558,-0.141369,-0.491125,41,2,0,1,0,0,0,0


# AdaBoost

In [8]:
df3

,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,감마 지티피,시도코드,성별코드,ast_alt2,E10,I10,J00,J20,M50
0,0.182520,-0.621876,-0.717914,0.831830,0.499025,1.426068,-0.253558,0.764726,-0.219147,27,1,0,0,1,0,0,0
1,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,44,1,0,0,0,1,0,0
2,1.457176,-0.621876,5.794160,-1.861579,1.328992,0.977745,-0.253558,-0.443400,-0.044303,44,1,0,0,0,0,1,0
3,-0.549760,-0.373717,-0.003174,-0.503164,-0.735804,-0.559361,-0.253558,-0.443400,0.033405,41,2,0,0,0,1,0,0
4,-0.361858,-0.311677,-0.479667,-1.088688,0.418053,-0.751499,-0.253558,-0.443400,-0.491125,41,2,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330953,0.804423,0.898099,-0.201713,-0.573427,-0.523251,0.721561,-0.253558,-0.443400,-0.413417,11,2,0,1,0,0,0,0
330954,1.458017,-0.311677,-0.241421,-1.322897,-0.128511,0.593469,-0.253558,-0.141369,-0.024876,30,1,0,0,0,1,0,0
330955,0.553416,0.308721,0.790981,-0.479743,1.693368,-0.367222,-0.253558,-0.745431,-0.374563,41,2,0,0,0,0,0,1
330956,-0.756504,-1.552472,-0.241421,2.096562,-0.411914,-0.111038,-0.253558,-0.141369,-0.355136,26,2,0,0,0,0,1,0


In [10]:
# E10
train_input, test_input, train_target, test_target = train_test_split(df3.iloc[:, :-5], df3.iloc[:,-5], test_size=0.2, random_state=42)

In [11]:
ada_model = AdaBoostClassifier(n_estimators=50, random_state=42)

# 모델 학습
ada_model.fit(train_input, train_target)

# 예측 수행
y_pred = ada_model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.9140530577713318


In [12]:
confusion_matrix(test_target, y_pred)

array([[60503,     0],
       [ 5689,     0]], dtype=int64)

# Adaboost + RF

In [15]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(class_weight={0: 1, 1: 5}, random_state=42)

ada_model = AdaBoostClassifier(base_estimator=model,
                               n_estimators=50,
                               random_state=42)

# 모델 학습
ada_model.fit(train_input, train_target)

# 예측 수행
y_pred = ada_model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

c:\Users\USER\anaconda3\envs\nltk\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


정확도: 0.8659958907420836


In [16]:
confusion_matrix(test_target, y_pred)

array([[57322,  3181],
       [ 5689,     0]], dtype=int64)

# AdaBoost + XGB

In [116]:
model = XGBClassifier(objective='binary:logistic',
                      max_depth = 7,
                      learning_rate = 1,
                      n_estimators=500,
                      scale_pos_weight = 11,
                      gamma = 1,
                      reg_lambda = 5,
                      reg_alpha = 0,
                      enable_categorical=True)

ada_model = AdaBoostClassifier(base_estimator=model,
                               n_estimators=100,
                               random_state=42)

# 모델 학습
ada_model.fit(train_input, train_target)

# 예측 수행
y_pred = ada_model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.6832698815566836


In [117]:
confusion_matrix(test_target, y_pred)

{'learning_rate': 1, 'reg_alpha': 0, 'reg_lambda': 5}

In [118]:
confusion_matrix(test_target, y_pred)

array([[43956, 16547],
       [ 4418,  1271]], dtype=int64)

#### scale_pos_weight값 계산

In [144]:
count = df3.iloc[:,-5].value_counts()
count[0] / count[1]

10.644020687471414

In [282]:
model = XGBClassifier(objective='binary:logistic',
                      max_depth = 7,
                      learning_rate = 1,
                      n_estimators=500,
                      scale_pos_weight = 11,
                      gamma = 1,
                      reg_lambda = 5,
                      reg_alpha = 0,
                      enable_categorical=True)

# 모델 학습
model.fit(train_input, train_target)

# 예측 수행
y_pred = model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.5865512448634276


# 튜닝_다운샘플링

In [263]:
train_input

,bmi,map,식전혈당(공복혈당),총 콜레스테롤,트리글리세라이드,혈색소,요단백,혈청크레아티닌,감마 지티피,시도코드,성별코드,ast_alt2
226254,1.432552,0.742999,3.213155,2.518139,-0.057660,-0.559361,1.723890,-0.745431,1.199028,47,2,0
72589,-0.829323,-0.063518,0.115949,-0.339217,0.235865,1.105837,-0.253558,1.066757,0.072259,45,1,0
87582,-0.601220,-0.528816,0.790981,-1.322897,-0.209483,1.362022,-0.253558,0.462694,-0.277428,41,1,0
25838,0.338024,0.277701,-0.122297,1.370512,0.519268,1.810345,1.723890,0.764726,-0.199720,11,1,0
69152,0.495197,0.401781,0.751274,1.534459,0.478782,1.297976,-0.253558,-0.141369,1.354444,27,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.804423,-2.203890,-0.519375,0.386832,-0.108268,-0.303177,-0.253558,-0.443400,-0.413417,11,2,0
259178,-0.086620,-0.435756,-0.003174,-0.315796,-0.806655,0.081100,-0.253558,0.160663,-0.296855,11,1,0
131932,-0.361858,0.153622,0.671858,-0.011324,-0.816776,-1.391960,-0.253558,1.066757,-0.335709,41,1,0
146867,-1.303767,-1.118194,-0.400252,1.370512,-0.786412,-1.007684,-0.253558,-0.443400,-0.510552,45,2,0


In [268]:
train_target.value_counts()

0    223189
1     41577
Name: I10, dtype: int64

In [281]:
from imblearn.over_sampling import RandomOverSampler

rus = RandomOverSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(train_input, train_target)

model = XGBClassifier(objective='binary:logistic',
                      max_depth = 20,
                      learning_rate = 0.1,
                      n_estimators=300,
                    #   scale_pos_weight = 11,
                      gamma = 1,
                      reg_lambda = 5,
                      reg_alpha = 0,
                      enable_categorical=True)

# 모델 학습
model.fit(X_resampled, y_resampled)

# 예측 수행
y_pred = model.predict(test_input)

# 정확도 계산
accuracy = accuracy_score(test_target, y_pred)
print(f'정확도: {accuracy}')

정확도: 0.7326565143824028


In [283]:
confusion_matrix(test_target, y_pred)

array([[36062, 19547],
       [ 7820,  2763]], dtype=int64)

# for문

In [172]:
j = 1
for i in range(-5, 0):
    train_input, test_input, train_target, test_target = train_test_split(df3.iloc[:, :-5], df3.iloc[:,i], test_size=0.2, random_state=42)

    count = df3.iloc[:,i].value_counts()
    num = round(count[0] / count[1])

    model = XGBClassifier(objective='binary:logistic',
                        max_depth = 7,
                        learning_rate = 1,
                        n_estimators=500,
                        scale_pos_weight = num,
                        gamma = 1,
                        reg_lambda = 5,
                        reg_alpha = 0,
                        enable_categorical=True)

    # 모델 학습
    model.fit(train_input, train_target)

    # 예측 수행
    y_pred = model.predict(test_input)

    # 정확도 계산
    accuracy = accuracy_score(test_target, y_pred)
    print(f'정확도: {accuracy}')
    
    print(confusion_matrix(test_target, y_pred))
    print('-'*50)
    globals()['model{}'.format(j)] = model
    j+=1

정확도: 0.7784324389654339
[[50893  9610]
 [ 5056   633]]
--------------------------------------------------
정확도: 0.6370256224317138
[[40013 15596]
 [ 8430  2153]]
--------------------------------------------------
정확도: 0.4988820401256949
[[25991 18978]
 [14192  7031]]
--------------------------------------------------
정확도: 0.471703529127387
[[23744 25302]
 [ 9667  7479]]
--------------------------------------------------
정확도: 0.5710810974135847
[[34800 19841]
 [ 8550  3001]]
--------------------------------------------------


# 범주해당 확률 변수 생성

In [187]:
for j in range(1,6):
    print(globals()['model{}'.format(j)].predict_proba(test_input.iloc[[1]])[0][0])

0.9974088
0.994545
0.95581627
0.6002408
0.08348602


In [246]:
result = []
for i in range(len(test_input)):
    val = test_input.iloc[[i]]
    temp = []
    for j in range(1,6):
        temp.append(globals()['model{}'.format(j)].predict_proba(val)[0][0])
    result.append(temp.index(max(temp)))
    
temp2 = df['상병기호'][test_target.index]

a = np.column_stack([temp2, result])
len(a[a[:,0] == a[:,1]])/len(a)

0.14111977278220933

In [215]:
import numpy as np
a = np.column_stack([test_target, result])
len(a[a[:,0] == a[:,1]])/len(a)

0.5391890258641527

In [219]:
result = []
for i in range(len(test_input)):
    val = test_input.iloc[[i]]
    temp = []
    for j in range(1,6):
        temp.append(globals()['model{}'.format(j)].predict_proba(val)[0][1])
    result.append(temp.index(max(temp)))

a = np.column_stack([test_target, result])
len(a[a[:,0] == a[:,1]])/len(a)

0.13001571186850375

In [239]:
test_target.value_counts()

0    54641
1    11551
Name: M50, dtype: int64

In [241]:
globals()['model{}'.format(j)].predict_proba(val)

array([[0.82950413, 0.17049588]], dtype=float32)

In [231]:
temp2 = df['상병기호'][test_target.index]

a = np.column_stack([temp2, result])
len(a[a[:,0] == a[:,1]])/len(a)

0.15911288373217308

In [221]:
from collections import Counter

# Counter를 사용하여 각 요소의 개수를 세기
counts = Counter(result)

# 각 값의 개수 출력
counts[4]

15036

In [217]:
import pickle
for j in range(1,6):
    with open('model{}.pickle'.format(j),'wb') as fw:
        pickle.dump(globals()['model{}'.format(j)], fw)